Este notebook tiene como fin ser un monitoreo de los datos mas recientes de gdelt, utiliza el archivo que se actualiza cada 15 minutos

En particular busca en el archivo más reciente si hay menciones de las entidades referentes al Food System Summit

In [51]:
import pandas as pd
from datetime import datetime, timedelta

In [60]:
import urllib.request as req
import zipfile
import os
import numpy as np

In [53]:
# crear objeto datetime en base al string de fecha
time_obj = datetime.strptime("2021-02-01-0000", '%Y-%m-%d-%H%M')

In [69]:
def get_date(datetime_obj):
    return "".join(datetime_obj.strftime(x) for x in ["%Y", "%m", "%d", "%H", "%M"]) + "00"

date = get_date(time_obj)

In [70]:
date

'20210201000000'

In [59]:
url = f"http://data.gdeltproject.org/gdeltv2/{date}.gkg.csv.zip"

In [61]:
filedata = req.urlopen(url)
datatowrite = filedata.read()
filename = url.split('/')[-1]
with open(f"real_time/{filename}", 'wb') as f:
    f.write(datatowrite)
with zipfile.ZipFile(f"real_time/{filename}","r") as zip_ref:
    zip_ref.extractall("real_time")

In [62]:
df = pd.read_csv(f"real_time/{filename[:-4]}", sep='\t', header=None, encoding="latin-1") # cols 3 link, 13 entities
df = df[~df[13].isnull()].reset_index(drop=True)

In [63]:
df[13]

0      centre for strategic studies;university of jor...
1      u s marine mammal commission;university of cal...
2                 instagram;guitar international;youtube
3                  oklahoma city thunder;houston rockets
4                               texas baptist university
                             ...                        
689                             athletic club in la liga
690    chicago teachers union;suder montessori magnet...
691                            pfizer;ministry of health
692    republican party;kinzinger future first leader...
693    school activities;university of akron;veterina...
Name: 13, Length: 694, dtype: object

In [64]:
os.remove(f"real_time/{filename}")
os.remove(f"real_time/{filename[:-4]}")

In [66]:
searched_entities = ["un-fss", "unfss", "fss", "food systems summit", "food system summit", r"un[ a-z0-9]*summit", r"food[ a-z0-9]*summit"]

In [67]:
found = False
for i in range(len(searched_entities)):
    found = found | df[13].str.contains(searched_entities[i], regex=True)

In [47]:
if found.any():
    with open("real_time/found_files.txt", 'w') as f:
        f.write(f"{date}\n")

False

In [71]:
time_obj += timedelta(minutes=15)

In [76]:
time_obj

datetime.datetime(2021, 2, 1, 0, 15)

In [73]:
date = get_date(time_obj)

'20210201001500'

basta con realizar un loop infinito sobre el codigo anterior para actualizar en tiempo real (15 min)